In this notebook Longitude and latitude columns are added in order for further visualisations and processing. This proccess take a lot of time, therefore it is done in separate notebook and columns added before all further manipulation with the dataset.

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

In [2]:
df = pd.read_parquet("../data/raw/pollution_dataset.parquet", engine="pyarrow")
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [ ]:
# Use only columns needed for geocoding
unique_addresses = df[["Address", "City", "State"]].drop_duplicates().copy()
print("Unique Address combinations:", len(unique_addresses))


In [ ]:
def clean_address(a):
    if not isinstance(a, str):
        return ""
    a = a.replace("STN", "")
    a = a.replace("-", " ")
    a = a.replace("  ", " ")
    return a.strip()

unique_addresses["CleanAddress"] = unique_addresses["Address"].apply(clean_address)

# Build full postal-style address
unique_addresses["FullAddress"] = (
    unique_addresses["CleanAddress"] + ", " +
    unique_addresses["City"].fillna("") + ", " +
    unique_addresses["State"].fillna("")
).str.strip()


In [ ]:
unique_addresses.head()


In [ ]:
geolocator = Nominatim(user_agent="aqi_geocoder_app")


In [ ]:
def geocode_address(addr):
    try:
        loc = geolocator.geocode(addr)
        if loc:
            return loc.latitude, loc.longitude
        return None, None
    except:
        return None, None
# Geocode with a delay to respect usage policy

In [ ]:
unique_addresses["Latitude"] = latitudes
unique_addresses["Longitude"] = longitudes


In [ ]:
unique_addresses.to_parquet("../data/raw/coordinates.parquet", index=False)
print("Saved coordinates.parquet")


In [6]:
unique_addresses.head()
# print(unique_addresses.isna().sum())
# print(unique_addresses.shape)

,Address,Latitude,Longitude
0,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,33.458116,-112.022742
1,2857 N MILLER RD-S SCOTTSDALE STN,NaN,NaN
2,"1237 S. BEVERLY , TUCSON",32.206263,-110.879259
3,2956-A TREAT BOULEVARD,37.936691,-122.028736
4,5551 BETHEL ISLAND RD,38.003272,-121.641333


In [7]:
print(unique_addresses.isna().sum())
print(unique_addresses.shape)

Address       0
Latitude     93
Longitude    93
dtype: int64
(204, 3)


In [8]:
failed = unique_addresses[unique_addresses["Latitude"].isna()]
failed

,Address,Latitude,Longitude
1,2857 N MILLER RD-S SCOTTSDALE STN,NaN,NaN
5,"UNIT 759 EL PORTAL SHOPPING CENTER, San Pablo",NaN,NaN
15,"7823 BLACKFOOT WAY, NORTH HIGHLANDS",NaN,NaN
16,"DEL PASO-2701 AVALON DR, SACRAMENTO",NaN,NaN
20,"1100 PASEO INTERNATIONAL, OTAY MESA, CA",NaN,NaN
...,...,...,...
190,"2730 CANNONS LANE, BOWMAN FIELD",NaN,NaN
191,"BLAIR STREET: 3247 Blair Street, St. Louis, MO...",NaN,NaN
198,"Piney Run, Frostburg Reservoir, Finzel",NaN,NaN
201,NARSTO SITE - ARENDTSVILLE,NaN,NaN


In [5]:
"""
df = pd.merge(df, unique_addresses, on="Address", how="left")
df.to_parquet("../data/preprocessed/pollution_dataset_enriched.parquet", index=False)
"""

'\ndf = pd.merge(df, unique_addresses, on="Address", how="left")\ndf.to_parquet("../data/preprocessed/pollution_dataset_enriched.parquet", index=False)\n'